***
# <font color=green size=10>Lista 1: geração de números pseudo-aleatórios</font>
***

In [1]:
# !pip install pandas
# !pip install numpy
# !pip install seaborn
# !pip install DeepSaki

▶ Importando bibliotecas

In [2]:
import DeepSaki
strategy, RUNTIME_ENVIRONMENT, hw_accelerator_handle = DeepSaki.utils.DetectHw()
print(strategy)

Running on single GPU  /device:GPU:0
Number of accelerators:  1
____________________________________________________________________________________
Device List: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10742012146249262507
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10047455232
locality {
  bus_id: 1
  links {
  }
}
incarnation: 509162452294179845
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import string

### Questão 1 - Simulando computacionalmente o gerador de Babel


▶ Importando e tratando o dicionário

In [4]:
dicionario = pd.read_csv("Dicionario.txt", names=["vocabulo"])
dicionario.vocabulo = dicionario.vocabulo.str.strip()
dicionario.vocabulo = dicionario.vocabulo.str.lower()
dicionario.drop_duplicates(inplace=True, ignore_index=True)
dicionario["tamanho"] = dicionario.vocabulo.str.len()

▶ Definindo parâmetros globais

In [5]:
# cria um dataset do alfabeto contendo a letra, se é vogal e a frequência da letra na língua portuguesa
alfabeto = pd.DataFrame(list(zip(list(string.ascii_lowercase),
                                 [1 if l in "aeiou" else 0 for l in list(string.ascii_lowercase)],
                                 [0.1463, 0.0104, 0.0388, 0.0499, 0.1257, 0.0102, 0.013, 0.0128,
                                  0.0618, 0.004, 0.0002, 0.0278, 0.0474, 0.0505, 0.1073, 0.0252,
                                  0.012, 0.0653, 0.0781, 0.0434, 0.0463, 0.0166,0.0001, 0.0021,
                                  0.0001, 0.0047])),
                        columns=["letra", "vogal", "frequencia"])
num_min_iteracoes = 10000
num_max_iteracoes = 2000000
step = 10000
# num_min_iteracoes = 100000
# num_max_iteracoes = 200000
# step = 100000
num_palavras_possiveis = (alfabeto.shape[0] ** 5)  # amostras ordenadas com reposição - (26^5)

▶ Definindo funções auxiliares

In [6]:
def gerar_seq_caracteres_aleatoria(tamanho_seq_carateres=5, vogais_consoantes_alternadas=False, p_uniforme=True):
    letras = alfabeto.letra.to_list()

    if vogais_consoantes_alternadas:
        # gera sequência de caracteres, alternando-se vogais e consoantes aleatórias

        # inicia variáveis
        vogais = alfabeto.query("vogal == 1").letra.to_list()
        consoantes = alfabeto.query("vogal == 0").letra.to_list()
        letra_corrente_vogal = False

        # gera a primeira letra da sequência de caracteres e registra se é uma vogal
        sequencia_caracteres = np.random.choice(letras)
        cont = 1
        if sequencia_caracteres in vogais:
            letra_corrente_vogal = True

        # gera as demais letras da sequência até atingir o tamanho estabelecido no parâmetro
        while cont < tamanho_seq_carateres:
            if letra_corrente_vogal:
                sequencia_caracteres += np.random.choice(consoantes)
                letra_corrente_vogal = False
            else:
                sequencia_caracteres += np.random.choice(vogais)
                letra_corrente_vogal = True
            cont += 1
    else:
        # gera sequência de carateres com 5 letras aleatoriamente
        p = None
        if not p_uniforme:
            # o parâmetro indica se deve ou não ser usada probabilidade de distribuição uniforme
            # caso não seja uniforme, distribui ajusta a probabilidade de acordo com a frequência do aparecimento
            # de cada uma das letras do alfabeto na língua portuguesa
            p = alfabeto.frequencia.to_list()

        while True:
            sequencia_caracteres = ''.join(np.random.choice(letras, size=tamanho_seq_carateres, replace=True, p=p))
            if (not p_uniforme and "a" in sequencia_caracteres) or p_uniforme:
                break

    return sequencia_caracteres

In [7]:
def simulacao_monte_carlo_palavras(dicionario, num_instancias, vogais_consoantes_alternadas, p_uniforme=True):
    palavras = []
    dicionario_lista = dicionario.vocabulo.to_list()
    # np.random.seed(42)
    for _ in range(num_instancias):
        sequencia_caracteres = gerar_seq_caracteres_aleatoria(vogais_consoantes_alternadas=vogais_consoantes_alternadas,
                                                              p_uniforme=p_uniforme)
        if sequencia_caracteres in dicionario_lista:
            palavras.append(sequencia_caracteres)

    return palavras

In [8]:
def executar_simulacoes(dicionario, vogais_consoantes_alternadas=False, p_uniforme=True):
    probabilidades = pd.DataFrame()

    for num_instancias_simulacao in range(num_min_iteracoes, num_max_iteracoes + 1, step):
        palavras_validas_geradas = simulacao_monte_carlo_palavras(dicionario=dicionario,
                                                                  num_instancias=num_instancias_simulacao,
                                                                  vogais_consoantes_alternadas=vogais_consoantes_alternadas,
                                                                  p_uniforme=p_uniforme)
        num_palavras_validas_geradas = len(palavras_validas_geradas)
        # número de palavras geradas aleatoriamente válidas / número de palavras geradas
        probabilidade_simulacao = num_palavras_validas_geradas / num_instancias_simulacao
        probabilidades = pd.concat([probabilidades,
                                    pd.DataFrame({"num_instancias": [num_instancias_simulacao],
                                                  "num_palavras_validas_geradas": [num_palavras_validas_geradas],
                                                  "probabilidade": [probabilidade_simulacao],
                                                  "palavras_validas_geradas": [palavras_validas_geradas]})],
                                   axis=0, ignore_index=True)

    return probabilidades

In [9]:
def plotar_grafico(probabilidades_estimadas):
    ax = sns.scatterplot(data=probabilidades_estimadas, x="num_instancias", y="probabilidade")
    ax.figure.set_size_inches(14, 6)
    ax.hlines(y=probabilidades_estimadas.probabilidade.mean(), xmin=0, xmax=probabilidades_estimadas.num_instancias.max(), colors='red', linestyles='dashed')

**a)** Estime via simulação computacional (Monte Carlo) a probabilidade de se gerar uma palavra válida
 (isso é, do dicionário) ao sortear ao acaso sequências de 5 letras (todas com a mesma probabilidade). Em
seguida, calcule analiticamente tal probabilidade e faça um gráfico indicando se a estimativa obtida se
aproxima do valor teórico conforme a amostra aumenta. **Atenção**: utilize somente as letras do alfabeto
sem carateres especiais.


▶ Derivando o dicionário apenas com palavras de 5 letras

In [10]:
dicionario_5_letras = dicionario.query("tamanho == 5")
dicionario_5_letras.head()

,vocabulo,tamanho
1,aarao,5
11,abaco,5
13,abade,5
19,abafa,5
69,abafe,5


▶ Obtendo as probabilidades e plotando o gráfico

In [11]:
try:
    probabilidades_estimadas_5_letras = pd.read_csv("probabilidades_estimadas_5_letras.csv")
except FileNotFoundError:
    probabilidades_estimadas_5_letras = executar_simulacoes(dicionario_5_letras)
    probabilidades_estimadas_5_letras.to_csv("probabilidades_estimadas_5_letras.csv", index=False)

probabilidades_estimadas_5_letras

,num_instancias,num_palavras_validas_geradas,probabilidade,palavras_validas_geradas
0,10000,7,0.000700,"['ronda', 'leigo', 'abati', 'iscas', 'cuspi', ..."
1,20000,9,0.000450,"['dures', 'afane', 'vossa', 'juiza', 'adule', ..."
2,30000,13,0.000433,"['corai', 'aarao', 'vazem', 'cofre', 'vosso', ..."
3,40000,16,0.000400,"['fitar', 'rugas', 'rejam', 'povoa', 'ligar', ..."
4,50000,21,0.000420,"['maior', 'usado', 'frijo', 'atemo', 'temam', ..."
...,...,...,...,...
195,1960000,917,0.000468,"['cessa', 'sutil', 'gerir', 'gemeo', 'solem', ..."
196,1970000,870,0.000442,"['lenta', 'gomas', 'luzes', 'ruias', 'acaso', ..."
197,1980000,905,0.000457,"['fisga', 'nessa', 'filia', 'jegue', 'envia', ..."
198,1990000,879,0.000442,"['longe', 'peite', 'situe', 'aorta', 'manco', ..."


In [12]:
print(f"Probabilidade média estimada das simulações: {probabilidades_estimadas_5_letras.probabilidade.mean() * 100:0.8f}%")

num_palavras_dicionario_5_letras = dicionario_5_letras.shape[0]
probabilidade_teorica_5_letras = num_palavras_dicionario_5_letras / num_palavras_possiveis
print(f"Probabilidade teórica de uma palavra válida de 5 letras: {probabilidade_teorica_5_letras * 100:0.8f}%")

Probabilidade média estimada das simulações: 0.04574322%
Probabilidade teórica de uma palavra válida de 5 letras: 0.04567653%


In [13]:
plotar_grafico(probabilidades_estimadas_5_letras)

**b)** Estime a probabilidade da sequência gerada ser um palíndromo (ou seja, pode ser lida, indiferentemente,
da esquerda para direita ou da direita para esquerda). Compare o resultado com a probabilidade exata,
calculada analiticamente.

▶ Obtendo o dicionário apenas com os palíndromos de 5 letras

In [14]:
dicionario_palindromo = dicionario_5_letras.loc[dicionario_5_letras.vocabulo == dicionario_5_letras.vocabulo.str[::-1]]
dicionario_palindromo.head()

,vocabulo,tamanho
7156,adida,5
8606,aerea,5
10082,afofa,5
17140,amima,5
24056,arara,5


In [15]:
try:
    probabilidades_estimadas_palindromo = pd.read_csv("probabilidades_estimadas_palindromo.csv")
except FileNotFoundError:
    probabilidades_estimadas_palindromo = pd.DataFrame()

    for num_instancias, _, _, palavras_validas_geradas in probabilidades_estimadas_5_letras.values:
        palindromos = []
        palavras_validas_geradas = str(palavras_validas_geradas) # se for do tipo list, transforma na representação str
        for palavra in eval(palavras_validas_geradas): # eval tranforma transforma uma representação str de uma lista no tipo list
            if palavra == palavra[::-1]:
                palindromos.append(palavra)

        probabilidade = len(palindromos) / num_instancias
        probabilidades_estimadas_palindromo = pd.concat([probabilidades_estimadas_palindromo,
                                                         pd.DataFrame([[num_instancias,
                                                                        len(palindromos),
                                                                        probabilidade,
                                                                        palindromos]],
                                                         columns=probabilidades_estimadas_5_letras.columns)],
                                                        ignore_index=True)
        probabilidades_estimadas_palindromo.to_csv("probabilidades_estimadas_palindromo.csv", index=False)

probabilidades_estimadas_palindromo

,num_instancias,num_palavras_validas_geradas,probabilidade,palavras_validas_geradas
0,10000,0,0.000000,[]
1,20000,0,0.000000,[]
2,30000,0,0.000000,[]
3,40000,0,0.000000,[]
4,50000,0,0.000000,[]
...,...,...,...,...
195,1960000,3,0.000002,"['sedes', 'solos', 'soros']"
196,1970000,8,0.000004,"['rajar', 'reter', 'mexem', 'sagas', 'saras', ..."
197,1980000,7,0.000004,"['mexem', 'raiar', 'melem', 'supus', 'solos', ..."
198,1990000,9,0.000005,"['socos', 'reter', 'metem', 'reger', 'mamam', ..."


In [16]:
print(f"Probabilidade média estimada das simulações: {probabilidades_estimadas_palindromo.probabilidade.mean() * 100:0.8f}%")

num_palavras_dicionario_palindromo = dicionario_palindromo.shape[0]
probabilidade_teorica_palindromo = num_palavras_dicionario_palindromo / num_palavras_possiveis
print(f"Probabilidade teórica de um palíndromo de 5 letras: {probabilidade_teorica_palindromo * 100:0.8f}%")

Probabilidade média estimada das simulações: 0.00033112%
Probabilidade teórica de um palíndromo de 5 letras: 0.00033666%


In [17]:
plotar_grafico(probabilidades_estimadas_palindromo)

<b>c)</b> Construa um gerador que alterne entre consoantes e vogais (se uma letra for uma vogal, a próxima será
uma consoante e vice-versa). Qual a probabilidade de gerar uma palavra válida com este novo gerador?

In [18]:
try:
    probabilidades_estimadas_vogais_consoantes_alternadas = pd.read_csv("probabilidades_estimadas_vogais_consoantes_alternadas.csv")
except FileNotFoundError:
    probabilidades_estimadas_vogais_consoantes_alternadas = executar_simulacoes(dicionario_5_letras, vogais_consoantes_alternadas=True)
    probabilidades_estimadas_vogais_consoantes_alternadas.to_csv("probabilidades_estimadas_vogais_consoantes_alternadas.csv", index=False)

probabilidades_estimadas_vogais_consoantes_alternadas


KeyboardInterrupt



In [ ]:
print(f"Probabilidade média estimada das simulações: {probabilidades_estimadas_vogais_consoantes_alternadas.probabilidade.mean() * 100:0.8f}%")

In [ ]:
plotar_grafico(probabilidades_estimadas_vogais_consoantes_alternadas)

**d)** Considere um processo gerador de sequências de 5 caracteres no qual cada letra é sorteada com
probabilidade proporcional à sua respectiva frequência na língua portuguesa (veja essa [página](https://pt.wikipedia.org/wiki/Frequ%C3%AAncia_de_letras)). Suponha
que esse processo gerou uma sequência com ao menos um “a”. Neste caso, estime a probabilidade dessa
sequência ser uma palavra válida.

In [ ]:
try:
    probabilidades_estimadas_frequencia_5_letras = pd.read_csv("probabilidades_estimadas_frequencia_5_letras.csv")
except FileNotFoundError:
    probabilidades_estimadas_frequencia_5_letras = executar_simulacoes(dicionario_5_letras, p_uniforme=False)
    probabilidades_estimadas_frequencia_5_letras.to_csv("probabilidades_estimadas_frequencia_5_letras.csv", index=False)

probabilidades_estimadas_frequencia_5_letras

In [ ]:
print(f"Probabilidade média estimada das simulações: {probabilidades_estimadas_frequencia_5_letras.probabilidade.mean() * 100:0.8f}%")

In [ ]:
plotar_grafico(probabilidades_estimadas_frequencia_5_letras)